In [1]:
from pyscf import scf,gto
import basis_set_exchange as bse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pyscf.geomopt import optimize


/home/giorgio/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.append("../Alchemical_CPHF_perturbator_pyscf22")
from FcMole import *
from AP_class import APDFT_perturbator as AP
from AP_utils import DeltaV
import copy
from AP_class import charge2symbol

def target_mol(mol,sites,pvec):
    splitted=(mol.atom.split())
    refchgs=copy.deepcopy(mol.atom_charges())
    for idx in range(len(pvec)):
        refchgs[sites[idx]]+=int(pvec[idx])
    for idx in range(len(pvec)):
        splitted[sites[idx]*4]=charge2symbol[refchgs[sites[idx]]]
    for i in range(3,len(splitted),4): 
        splitted[i]+=";"
    atomstr=" ".join(splitted)
    tmol=gto.M(atom=atomstr,unit=mol.unit,basis=mol.basis,charge=0,verbose=0)#mol.charge+sum(pvec))
    return tmol

In [3]:
basis_pcx2={"H":"pc-2",'B':bse.get_basis("pcX-2",fmt="nwchem",elements=[5])\
            ,'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])\
           ,'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])\
           ,'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])\
           ,'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])}

In [4]:
# n2- co,bf
# CO- N2,BF

In [5]:
#NN_nonopt=gto.M(atom="N 0 0 0; N 0 0 2.",basis=basis_pcx2,unit="au")
#NN_nonopt=gto.M(atom="NNopt_pcx2.xyz",basis=basis_pcx2)
#mf=scf.RKS(NN_nonopt)
#mf.xc="PBE0"
#NN=optimize(mf) #246435117496
#NN.tofile("NNopt_pcx2.xyz")


In [6]:
NN=gto.M(atom="NNopt_pcx2.xyz",basis=basis_pcx2)


In [7]:
#CO_nonopt=gto.M(atom="C 0 0 0; O 0 0 2.",basis=basis_pcx2,unit="au")
#mf=scf.RKS(CO_nonopt)
#mf.xc="PBE0"
#CO=optimize(mf)
#CO.tofile("COopt_pcx2.xyz")

In [8]:
CO=gto.M(atom="COopt_pcx2.xyz",basis=basis_pcx2)


In [9]:
#BF_nonopt=gto.M(atom="B 0 0 0; F 0 0 2.1",basis=basis_pcx2,unit="au")
#mf=scf.RKS(BF_nonopt)
#mf.xc="PBE0"
#BF=optimize(mf)
#BF.tofile("BFopt_pcx2.xyz")

In [10]:
BF=gto.M(atom="BFopt_pcx2.xyz",basis=basis_pcx2)


In [11]:
mf_NN=scf.RKS(NN)
mf_NN.xc="PBE0"
mf_NN.scf()
ap_nn=AP(mf_NN,sites=[0,1])
ap_nn.build_all()
P_nn=mf_NN.make_rdm1()

converged SCF energy = -109.445602751954


In [12]:
mf_CO=scf.RKS(CO)
mf_CO.xc="PBE0"
mf_CO.scf()
ap_co=AP(mf_CO,sites=[0,1])
ap_co.build_all()
P_co=mf_CO.make_rdm1()

converged SCF energy = -113.232052500924


In [13]:
mf_BF=scf.RKS(BF)
mf_BF.xc="PBE0"
mf_BF.scf()
ap_bf=AP(mf_BF,sites=[0,1])
ap_bf.build_all()
P_bf=mf_BF.make_rdm1()

converged SCF energy = -124.581813880113


In [14]:
APDFT1={}
APDFT2={}
APDFT3={}
half_appr={}
Levy_appr={}

target_energies={}

mf=mf_NN
mol=NN
ap=ap_nn

E0=mf.e_tot
E0_nuc=mf.energy_nuc()

t=np.array([1,-1])
label="NN->CO"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm)
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t

t=np.array([2,-2])
label="NN->BF"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t


/home/giorgio/anaconda3/lib/python3.8/site-packages/pyscf/gto/mole.py:1209: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)
/home/giorgio/anaconda3/lib/python3.8/site-packages/pyscf/gto/mole.py:1209: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -113.223015640727
converged SCF energy = -110.390567218457
converged SCF energy = -124.512367313606
converged SCF energy = -113.223015640727


In [15]:
mf=mf_CO
mol=CO
ap=ap_co

E0=mf.e_tot
E0_nuc=mf.energy_nuc()


t=np.array([1,-1])
label="CO->NN"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t


t=np.array([-1,1])
label="CO->BF"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t


converged SCF energy = -109.442588800006
converged SCF energy = -110.391631681503
converged SCF energy = -124.550872407919
converged SCF energy = -117.956330627089


In [16]:
mf=mf_BF
mol=BF
ap=ap_bf

E0=mf.e_tot
E0_nuc=mf.energy_nuc()


t=np.array([1,-1])
label="BF->CO"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t


t=np.array([2,-2])
label="BF->NN"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t
Levy_appr[label]=E0 +np.einsum("ij,ij",(mf.make_rdm1()+ t_scf.make_rdm1())/2,dV)-E0_nuc+enuc_t


converged SCF energy = -113.198590764638
converged SCF energy = -117.951842432886
converged SCF energy = -109.381530208655
converged SCF energy = -113.198590764638


In [17]:
df=pd.DataFrame()
df["APDFT1"]=pd.Series(APDFT1)
df["APDFT2"]=pd.Series(APDFT2)
df["APDFT3"]=pd.Series(APDFT3)
df["Target"]=pd.Series(target_energies)
df["Half aprrox"]=pd.Series(half_appr)
df["Levy aprrox"]=pd.Series(Levy_appr)

df

,APDFT1,APDFT2,APDFT3,Target,Half aprrox,Levy aprrox
NN->CO,-109.445603,-113.226247,-113.226247,-113.223016,-113.224667,-113.219676
NN->BF,-109.445603,-124.568180,-124.568180,-124.512367,-124.541894,-124.452724
CO->NN,-105.663887,-109.432638,-109.448297,-109.442589,-109.441003,-109.445789
CO->BF,-120.800218,-124.568970,-124.553312,-124.550872,-124.556466,-124.539740
BF->CO,-109.459918,-113.167302,-113.205196,-113.198591,-113.191731,-113.212197
BF->NN,-94.338022,-109.167560,-109.470708,-109.381530,-109.347137,-109.450868


In [18]:
error=pd.DataFrame()
error["APDFT1"] =(df["APDFT1"]-df.Target)*1000
error["APDFT2"] =(df["APDFT2"]-df.Target)*1000
error["APDFT3"] =(df["APDFT3"]-df.Target)*1000
error["Half aprrox"] =(df["Half aprrox"]-df.Target)*1000
error["Levy aprrox"] =(df["Levy aprrox"]-df.Target)*1000



error

,APDFT1,APDFT2,APDFT3,Half aprrox,Levy aprrox
NN->CO,3777.412889,-3.231490,-3.231490,-1.651713,3.340082
NN->BF,15066.764562,-55.812955,-55.812955,-29.526667,59.643445
CO->NN,3778.702231,9.950378,-5.708354,1.585541,-3.199817
CO->BF,3750.653975,-18.097879,-2.439147,-5.593871,11.132066
BF->CO,3738.672917,31.288269,-6.605177,6.860233,-13.605874
BF->NN,15043.508393,213.969804,-89.177764,34.393229,-69.338043


In [19]:
df.to_pickle("NNCOBF.pkl")
error.to_pickle("NNCOBF_error.pkl")

In [20]:
df.to_csv("NNCOBF.csv")
error.to_csv("NNCOBF_error.csv")

In [23]:
print(error.to_latex(float_format="{:0.2f}".format))

\begin{tabular}{lrrrrr}
\toprule
{} &   APDFT1 &  APDFT2 &  APDFT3 &  Half aprrox &  Levy aprrox \\
\midrule
NN->CO &  3777.41 &   -3.23 &   -3.23 &        -1.65 &         3.34 \\
NN->BF & 15066.76 &  -55.81 &  -55.81 &       -29.53 &        59.64 \\
CO->NN &  3778.70 &    9.95 &   -5.71 &         1.59 &        -3.20 \\
CO->BF &  3750.65 &  -18.10 &   -2.44 &        -5.59 &        11.13 \\
BF->CO &  3738.67 &   31.29 &   -6.61 &         6.86 &       -13.61 \\
BF->NN & 15043.51 &  213.97 &  -89.18 &        34.39 &       -69.34 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_15256/3657086161.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(error.to_latex(float_format="{:0.2f}".format))
